# Modelagem preditiva
_Machine Learning_

---

## Sumário

1. **Importação de bibliotecas**
2. **Carregamento das bases**
    - 2.1. Carregamento dos dataframes
    - 2.2. Extração de amostra dos dataframes
3. **Preparação dos dados**
    - 3.1. Exibição dos metadados
    - 3.2. Preparação das bases de treino e teste
        - 3.2.1. Análise de cardinalidade
        - 3.2.2. Variáveis não aplicáveis à modelagem
        - 3.2.3. Segmentação das bases **train** e **test**
        - 3.2.4. Transformação das features das bases **train** e **test**
4. **Modelagem preditiva**


<br>

---

<br>

## 1. Importação de bibliotecas

In [1]:
# Importação de pacotes e definição de parâmetros globais

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import time
import optuna

from pathlib import Path

from sklearn.model_selection import train_test_split

from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

d:\PROJETOS\deteccao_de_fraude_em_transacoes_financeiras\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configurações para exibição de dados no Jupyter Notebook

# Configurar para exibir todas as colunas do Dataframe
pd.set_option('display.max_columns', None)

# Configurar para exibir o conteúdo completo das colunas
pd.set_option('display.max_colwidth', None)

# Configurar a supressão de mensagens de aviso durante a execução
warnings.filterwarnings('ignore')

# Configurar o estilo do Seaborn
sns.set(style='whitegrid')

## 2. Carregamento das bases

### 2.1. Carregamento dos dataframes

In [3]:
# Efetuando a limpeza de memória antes do carregamento dos dados

print(f'\nQuantidade de objetos removidos da memória: {gc.collect()}')


Quantidade de objetos removidos da memória: 0


In [4]:
# Caminho base
base_path = Path('dados/dados_transformados_parquet/')

# Mapeamento nome_variavel: nome_arquivo
files = {
    'df_train': 'df_train.parquet',
    'df_val': 'df_val.parquet',
    'df_test': 'df_test.parquet'
}

# Carrega os DataFrames no dicionário
temp_dfs = {}
for var_name, file_name in files.items():
    try:
        temp_dfs[var_name] = pd.read_parquet(base_path / file_name)
    except Exception as e:
        print(f'Erro ao carregar {file_name}: {e}')

# Atribui às variáveis explicitamente
df_train_full, df_val_full, df_test_full = [temp_dfs[name] for name in ['df_train', 'df_val', 'df_test']]


In [5]:
# Exibindo da volumetria dos dados

print('\nVOLUMETRIA')
for name, df in temp_dfs.items():
    print(f'\n{name}')
    print('-' * 45)
    print(f'Quantidade de linhas (registros):  {df.shape[0]:,}')
    print(f'Quantidade de colunas (variáveis): {df.shape[1]:,}')



VOLUMETRIA

df_train
---------------------------------------------
Quantidade de linhas (registros):  3,566,068
Quantidade de colunas (variáveis): 157

df_val
---------------------------------------------
Quantidade de linhas (registros):  891,518
Quantidade de colunas (variáveis): 157

df_test
---------------------------------------------
Quantidade de linhas (registros):  2,194,300
Quantidade de colunas (variáveis): 157


### 2.2. Extração de amostra dos dataframes

In [6]:
# Função para geração de uma amostra estratificada de um DataFrame

def stratified_sample_classification(df, y_col, frac):
    '''
    Retorna uma amostra estratificada de um DataFrame com base na variável-alvo 
    mantendo a proporção original das classes.

    :param df: DataFrame
        DataFrame contendo as features e a variável-alvo.
    :param y_col: str
        Nome da coluna correspondente à variável-alvo (classe).
    :param frac: float
        Fração (entre 0 e 1) de registros a serem amostrados.
    :return: DataFrame
        Subconjunto amostrado do DataFrame original.
    '''
    sample_df, _ = train_test_split(
        df,
        train_size=frac,
        stratify=df[y_col],
        random_state=42
    )

    return sample_df

In [7]:
# Nome da variável-alvo
y_col = 'is_fraud'

# Fração desejada
frac = 0.2

# Gerando amostras estratificadas e uma amostra aleatória
df_train = stratified_sample_classification(df_train_full, y_col, frac)
df_val   = stratified_sample_classification(df_val_full, y_col, frac)
df_test  = df_test_full.sample(frac=frac, random_state=42)

In [8]:
# Comparativo dos tamanhos dos DataFrames originais e amostrados
print(f'       Original → Amostra')
print(f'Train: {len(df_train_full)}  → {len(df_train)}')
print(f'Val:   {len(df_val_full)}   → {len(df_val)}')
print(f'Test:  {len(df_test_full)}  → {len(df_test)}')

       Original → Amostra
Train: 3566068  → 713213
Val:   891518   → 178303
Test:  2194300  → 438860


## 3. Preparação dos dados

### 3.1. Exibição dos metadados

In [9]:
# Efetuando a limpeza de memória antes da exibição dos metadados

print(f'\nQuantidade de objetos removidos da memória: {gc.collect()}')


Quantidade de objetos removidos da memória: 0


In [10]:
# Função para geração de um dataframe de metadados

def generate_metadata(dataframe):
    '''
    Gera um DataFrame contendo metadados das colunas do DataFrame fornecido.

    :param dataframe: DataFrame
        DataFrame para o qual os metadados serão gerados.
    :return: DataFrame
        DataFrame contendo os metadados.
    '''
    metadata = pd.DataFrame({
        'Variável': dataframe.columns,
        'Tipo': dataframe.dtypes,
        'Qtde de nulos': dataframe.isnull().sum(),
        '% de nulos': round((dataframe.isnull().sum() / len(dataframe)) * 100, 2),
        'Cardinalidade': dataframe.nunique(),
    }).sort_values(by='Qtde de nulos', ascending=False).reset_index(drop=True)

    return metadata


In [11]:
# Exibe todas as linhas temporariamente

with pd.option_context('display.max_rows', None):
    display(generate_metadata(df_train))


,Variável,Tipo,Qtde de nulos,% de nulos,Cardinalidade
0,errors,object,701677,98.38,19
1,card_std_last_1h,float64,666036,93.39,27626
2,card_std_last_2h,float64,651574,91.36,34742
3,client_std_last_1h,float64,641749,89.98,38224
4,card_std_last_4h,float64,627914,88.04,45230
5,client_std_last_2h,float64,612370,85.86,50930
6,card_std_last_8h,float64,590553,82.80,61468
7,client_std_last_4h,float64,565953,79.35,71261
8,card_std_last_12h,float64,562257,78.83,74812
9,client_std_last_8h,float64,496414,69.60,106287


### 3.2. Preparação das bases de treino e teste

#### 3.2.1. Análise de cardinalidade

In [12]:
# Listando variáveis com determinado valor de cardinalidade

def list_low_cardinality_columns(dataframe, threshold=2):
    '''
    Lista os nomes das colunas de um DataFrame com cardinalidade inferior a um valor dado.

    :param dataframe: pd.DataFrame
        DataFrame a ser analisado.
    :param threshold: int
        Valor de corte para cardinalidade. Default é 2.
    :return: list
        Lista com os nomes das colunas cuja cardinalidade é menor que o threshold.
    '''
    return [col for col in dataframe.columns if dataframe[col].nunique(dropna=False) < threshold]


In [13]:
print('df_train - variáveis com baixa cardinalidade:\n', list_low_cardinality_columns(df_train))
print('\ndf_val - variáveis com baixa cardinalidade:\n', list_low_cardinality_columns(df_val))
print('\ndf_test - variáveis com baixa cardinalidade:\n', list_low_cardinality_columns(df_test))

df_train - variáveis com baixa cardinalidade:
 ['card_on_dark_web', 'flag_risky_chip_use', 'flag_no_chip_darkweb']

df_val - variáveis com baixa cardinalidade:
 ['card_on_dark_web', 'flag_risky_chip_use', 'flag_no_chip_darkweb']

df_test - variáveis com baixa cardinalidade:
 ['card_on_dark_web', 'transaction_id', 'is_fraud', 'flag_risky_chip_use', 'flag_no_chip_darkweb']


#### 3.2.2. Variáveis não aplicáveis à modelagem

In [14]:
# Lista as features de um DataFrame que contêm o termo 'id' no nome.

def list_columns_with_id(dataframe, case_sensitive=False):
    '''
    Lista os nomes das colunas de um DataFrame que contêm 'id' no nome.

    :param dataframe: pd.DataFrame
        DataFrame a ser analisado.
    :param case_sensitive: bool
        Se True, diferencia maiúsculas de minúsculas. Default é False.
    :return: list
        Lista com os nomes das colunas que contêm 'id'.
    '''
    if case_sensitive:
        return [col for col in dataframe.columns if 'id' in col]
    else:
        return [col for col in dataframe.columns if 'id' in col.lower()]


In [15]:
print('df_train - variáveis com termo "id":\n', list_columns_with_id(df_train))
print('\ndf_val - variáveis com termo "id":\n', list_columns_with_id(df_val))
print('\ndf_test - variáveis com termo "id":\n', list_columns_with_id(df_test))

df_train - variáveis com termo "id":
 ['id', 'client_id', 'card_id', 'merchant_id', 'id_card', 'client_id_card', 'id_client', 'transaction_id', 'merchant_id_is_new']

df_val - variáveis com termo "id":
 ['id', 'client_id', 'card_id', 'merchant_id', 'id_card', 'client_id_card', 'id_client', 'transaction_id', 'merchant_id_is_new']

df_test - variáveis com termo "id":
 ['id', 'client_id', 'card_id', 'merchant_id', 'id_card', 'client_id_card', 'id_client', 'transaction_id', 'merchant_id_is_new']


In [16]:
# Criando conjunto de variáveis que serão removidas dos DataFrames

vars_to_remove = ['zip', 'date', 'expires', 'acct_open_date']
vars_to_remove = vars_to_remove + ['id', 'client_id', 'card_id', 'merchant_id', 'id_card', 'client_id_card', 
                  'id_client', 'transaction_id', 'merchant_id_is_new','card_on_dark_web', 
                  'flag_risky_chip_use', 'flag_no_chip_darkweb']

#### 3.2.3. Segmentação das bases **train** e **test**

In [17]:
# Separando as variáveis preditivas e a variável preditora (alvo)

features = df_train.columns.drop('is_fraud')
features = features.drop(vars_to_remove, errors='ignore')
target = 'is_fraud'

In [18]:
# Separando as variáveis numéricas e categóricas

numerical_features = df_train[features].select_dtypes(exclude=[object, 'category']).columns
categorical_features = df_train[features].select_dtypes(include=[object, 'category']).columns

In [19]:
# Converter todas as colunas categóricas para string

df_train[categorical_features] = df_train[categorical_features].astype(str)
df_val[categorical_features] = df_val[categorical_features].astype(str)
df_test[categorical_features] = df_test[categorical_features].astype(str)

In [20]:
# Removendo as variáveis dos DataFrames de treino e teste

X_train = df_train[features]
y_train = df_train[target]

X_test = df_val[features]
y_test = df_val[target]

#### 3.2.4. Transformação das features das bases **train** e **test**

In [29]:
# Criando o pipeline para as variáveis numéricas
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

# Criando o pipeline para as variáveis categóricas
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', TargetEncoder())
])

In [30]:
# Criando o pipeline de pré-processamento que aplica transformações

preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, categorical_features),
    ('num', num_pipeline, numerical_features)
])
preprocessor = Pipeline(steps=[('preprocessor', preprocessor)])

In [31]:
# Verificando os shapes antes da transformação

print(f'Shape: X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'Shape: X_test:  {X_test.shape},  y_test: {y_test.shape}')

Shape: X_train: (713213, 140), y_train: (713213,)
Shape: X_test:  (178303, 140),  y_test: (178303,)


In [ ]:
# Aplicando o pré-processamento nos conjuntos de treino e teste

X_train_processed = preprocessor.fit_transform(X_train, y_train)
X_test_processed = preprocessor.transform(X_test)

In [33]:
# Verificando os shapes após transformação

print(f'Shape de X_train após transformação: {X_train_processed.shape}')
print(f'Shape de X_test após transformação:  {X_test_processed.shape}')

Shape de X_train após transformação: (713213, 140)
Shape de X_test após transformação:  (178303, 140)
